In [12]:
#!pip install transformers

In [13]:
from transformers import VisionEncoderDecoderModel, ViTFeatureExtractor, AutoTokenizer
import torch
from PIL import Image
import matplotlib.pyplot as plt
import cv2
import numpy as np

In [14]:
model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

In [15]:
feature_extractor = ViTFeatureExtractor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

In [16]:
tokenizer = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

max_length = 16
num_beams = 4
gen_kwargs = {"max_length": max_length, "num_beams": num_beams}
def load_and_predict(image_paths):
  images = []
  for image_path in image_paths:
    i_image = Image.open(image_path)
    if i_image.mode != "RGB":
      i_image = i_image.convert(mode="RGB")
    images.append(i_image)
  return predict(images)
def predict(images):
  pixel_values = feature_extractor(images=images, return_tensors="pt").pixel_values
  pixel_values = pixel_values.to(device)

  output_ids = model.generate(pixel_values, **gen_kwargs)

  preds = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
  preds = [pred.strip() for pred in preds]
  return preds

In [18]:
#test = ['../../../data/Cat.jpg', '../../../data/Puppies.jpg', '../../../data/Chess.jpg', '../../../data/Sarita.jpg']
test = ['../../../data/Sarita.jpg']
load_and_predict(test)

['a man and a woman dressed up in costumes']

In [19]:
cam = cv2.VideoCapture(0)
while True:
    val, img = cam.read()
    cv2.imshow("Image funcion", img)
    v = cv2.waitKey(1) & 0xFF
    if v == ord('q'): break
    elif v == ord('a'): print(predict(img))
cam.release()
cv2.destroyAllWindows()

['a man in a suit and tie']
